# Interactive Query with Gremlin
Gremlin is one of the most popular query languages in graph databases, as SQL in relational databases. Here, we will give some examples to illustrate how gremlin helps navigate the vertices and edges of a graph.

## Dataset
MODERN, a toy graph from tinkerpop, which consists of 6 vertices and 6 edges. We extend it to larger size by adding more complex relationships among vertices. Here are the edges:

\[(1,3),(1,2),(1,4),(4,5),(4,3),(6,3),(2,4),(4,1)\]

In [1]:
import os
import graphscope
from graphscope.framework.graph import Graph
from graphscope.framework.loader import Loader
import vineyard

k8s_volumes = {
    "data": {
        "type": "hostPath",
        "field": {
          "path": "/home/jovyan/datasets/modern_graph_2",  # Path in host
          "type": "Directory"
        },
        "mounts": {
          "mountPath": "/home/jovyan/datasets/modern_graph_2"  # Path in pods
          "readOnly": True
        }
    }
}
graphscope.set_option(show_log=True)  # enable logging
session = graphscope.session(k8s_volumes=k8s_volumes)  # create a session
modern_graph = session.load_from(
    vertices={
        "person": (Loader("/home/jovyan/datasets/modern_graph_2/person.csv", delimiter="|", header_row=True), ["name", ("age", "int")], "id"),
        "software": (Loader("/home/jovyan/datasets/modern_graph_2/software.csv", delimiter="|", header_row=True), ["name", "lang"], "id"),
    },
    edges={
        "knows": [Loader("/home/jovyan/datasets/modern_graph_2/knows.csv", delimiter="|"), [], (0, "person"), (1, "person")],
        "created": [Loader("/home/jovyan/datasets/modern_graph_2/created.csv", delimiter="|"), [], (0, "person"), (1, "software")],
    }, generate_eid=False)
interactive = session.gremlin(modern_graph)

2021-01-29 15:17:32,311 [INFO][session:377]: Initializing graphscope session with parameters: {'num_workers': 2, 'k8s_namespace': None, 'k8s_service_type': 'NodePort', 'k8s_gs_image': 'registry.cn-hongkong.aliyuncs.com/graphscope/graphscope:0.1.2', 'k8s_etcd_image': 'quay.io/coreos/etcd:v3.4.13', 'k8s_image_pull_policy': 'IfNotPresent', 'k8s_image_pull_secrets': [], 'k8s_gie_graph_manager_image': 'registry.cn-hongkong.aliyuncs.com/graphscope/maxgraph_standalone_manager:0.1.2', 'k8s_zookeeper_image': 'zookeeper:3.4.14', 'k8s_coordinator_cpu': 1.0, 'k8s_coordinator_mem': '4Gi', 'k8s_etcd_cpu': 0.5, 'k8s_etcd_mem': '128Mi', 'k8s_zookeeper_cpu': 0.5, 'k8s_zookeeper_mem': '128Mi', 'k8s_gie_graph_manager_cpu': 1.0, 'k8s_gie_graph_manager_mem': '4Gi', 'k8s_vineyard_cpu': 0.5, 'k8s_vineyard_mem': '512Mi', 'k8s_vineyard_shared_mem': '4Gi', 'k8s_engine_cpu': 0.5, 'k8s_engine_mem': '4Gi', 'k8s_volumes': {'data': {'type': 'hostPath', 'field': {'path': '/home/jovyan/datasets/modern_graph_2', 'type'

## Between Vertices
Traversals between two particular vertices is quite common situations in graph databases. For example, to figure out the relationships between v1 and v2/v3, a gremlin query can be written like this:

In [3]:
q1 = interactive.execute("g.V().has(\"id\", 1).as(\"u\").out().has(\"id\", eq(2).or(eq(3))).as(\"v\").select(\"u\", \"v\").by(\"id\")")
for p in q1:
    print(p)

[{'u': 1, 'v': 3}, {'u': 1, 'v': 2}]


Here is an example which is popular in social network scenarios, such as finding common features between two different people, one called "marko" while another called "peter".

In [4]:
q2 = interactive.execute("g.V().has(\"name\", \"marko\").out().where(__.in().has(\"name\", \"peter\")).valueMap()")
for p in q2:
    print(p)

[{'name': ['lop'], 'lang': ['java'], 'id': [3]}]


## Degree Centrality
Degree centrality is a measure of the number of edges associated to each vertex, which is of statistical significance in large-scale data processing. Here are some examples:

In [5]:
q3 = interactive.execute("g.V().group().by().by(bothE().count())")
for p in q3:
    print(p[0])
q4 = interactive.execute("g.V().group().by().by(inE().count())")
for p in q4:
    print(p[0])

{v[0.0]: 5, v[0.1]: 2, v[1.-4611686018427387904]: 3, v[0.2]: 1, v[1.-4611686018427387903]: 1, v[0.-9223372036854775808]: 4}
{v[0.0]: 2, v[1.-4611686018427387904]: 3, v[0.1]: 1, v[0.2]: 0, v[0.-9223372036854775808]: 1, v[1.-4611686018427387903]: 1}


## Cycle Detection
Cycle detection is another important application of graph query in commerce area where cycles are usually considered as fraudulent patterns. Here is an example illustrating how gremlin helps detect cycles in a graph.

In [6]:
q5 = interactive.execute("g.V().as(\"u\").repeat(out().simplePath()).times(2).where(out().where(eq(\"u\"))).count()")
print(q5.one())

[3]
